<a href="https://colab.research.google.com/github/Kauan-Messias/Desafio-Bravium-Analise-Sentimento/blob/master/modelo_classificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Modelo de classificação de sentimento**

Importando Bibliotecas: **'pandas', 'scikit-learn', 'nltk', 're', 'numpy'**

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
#nltk.download('stopwords')
#nltk.download('punkt_tab')

Importando Modelos

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from google.colab import drive
#drive.mount('/content/drive')

**Caminho do arquivo 'olist_order_reviews_dataset.csv' [Brazilian E-Commerce Public Dataset by Olist](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce?select=olist_order_reviews_dataset.csv)**

In [ ]:
caminho_arquivo = r'/content/drive/MyDrive/colab/olist_order_reviews_dataset.csv'

Carregamento e pré-processamentos de dados

In [ ]:
print("\nCarregamento e pré-processamentos de dados...\n")
try:
    reviews_df = pd.read_csv(caminho_arquivo)

except FileNotFoundError:
    print(f"\nERRO: O arquivo '{caminho_arquivo}' não foi encontrado.")
    print("Por favor, verifique se o caminho do arquivo esta correto.\n")
    exit()


Carregamento e pré-processamentos de dados...



Classificando notas positivo e negativo

In [ ]:
def classificar_sentimento(nota):
    if nota >= 4:
        return 'positivo'
    elif nota <= 2:
        return 'negativo'
    else:
        return 'neutro'

Aplicando classificação e filtrando sentimentos

In [ ]:
reviews_df['Sentimentos'] = reviews_df['review_score'].apply(classificar_sentimento)
reviews_df_filtrado = reviews_df[reviews_df['Sentimentos'] != 'neutro'].copy()
reviews_df_filtrado = reviews_df_filtrado.reset_index(drop = True)

Testes

In [ ]:
print(f"Total de reviews usadas: '{len(reviews_df_filtrado)}'\n")
print(reviews_df_filtrado['Sentimentos'].value_counts())
print()

Total de reviews usadas: '91045'

Sentimentos
positivo    76470
negativo    14575
Name: count, dtype: int64



Lista de **'stopwords'** em portugues

In [ ]:
stop_words_pt = set(stopwords.words('portuguese'))

Limpando textos

In [ ]:
def limpar_texto(texto):
    if not isinstance(texto, str):
        return ''

    #Converter letras para minusculo
    texto = texto.lower()

    #Remover caracteres especias e numeros
    texto = re.sub(r'[^a-z\s]', '', texto)

    #tokeniza palavras
    tokens = word_tokenize(texto)

    #remove stopwords
    tokens_limpos = [token for token in tokens if token not in stop_words_pt]

    #Junta tokens em uma string
    return ' '.join(tokens_limpos)

In [ ]:
print("\nLimpando Texto...\n")


Limpando Texto...



Aplicando limpeza em textos

In [ ]:
reviews_df_filtrado['review_comment_message_limpo'] = reviews_df_filtrado['review_comment_message'].apply(limpar_texto)

#Limpando linhas com valores nulos
reviews_df_filtrado = reviews_df_filtrado.dropna(subset = ['review_comment_message_limpo'])

#Limpando espaços
reviews_df_filtrado = reviews_df_filtrado[reviews_df_filtrado['review_comment_message_limpo'].str.strip() != '']
reviews_df_filtrado = reviews_df_filtrado.reset_index(drop = True)

In [ ]:
print("\nCarregamento e pré-processamentos de dados...\n")


Carregamento e pré-processamentos de dados...



 Definindo a variável de entrada e a variável alvo

In [ ]:

X = reviews_df_filtrado['review_comment_message_limpo']
y = reviews_df_filtrado['Sentimentos']

Dividindo dados para treino e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

Vetorizando texto usando TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(max_features = 5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
print("\nTreinando modelo de regressão logistíca e avaliando o desempenho...\n")


Treinando modelo de regressão logistíca e avaliando o desempenho...



Treinondo modelo

In [ ]:
model = LogisticRegression(max_iter = 1000)
model.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=1000)

Previsão e avaliação de modelo

In [ ]:
y_pred = model.predict(X_test_tfidf)

Testes: Acuracia e Relatório de Classificação

In [ ]:
print('Acurácia do modelo: ', accuracy_score(y_test, y_pred))
print('\nRelatório de Classificação: \n', classification_report(y_test, y_pred))

Acurácia do modelo:  0.9176739188826215

Relatório de Classificação: 
               precision    recall  f1-score   support

    negativo       0.86      0.86      0.86      2175
    positivo       0.94      0.94      0.94      5271

    accuracy                           0.92      7446
   macro avg       0.90      0.90      0.90      7446
weighted avg       0.92      0.92      0.92      7446



In [ ]:
print("\nExtraindo as palavras mais importantes para cada sentimento...\n")


Extraindo as palavras mais importantes para cada sentimento...



Extraindo Palavras

In [ ]:
feature_names = vectorizer.get_feature_names_out()
coefficients = model.coef_[0]

Criando DF view

In [ ]:
coef_df = pd.DataFrame({'Palavras:': feature_names,'Coef': coefficients})

Classificando 20 Palavras pelo coeficiente (Positivas e Negativas)

In [ ]:
top_positive_words = coef_df.sort_values(by='Coef', ascending=False).head(20)
top_negative_words = coef_df.sort_values(by='Coef', ascending=True).head(20)

In [ ]:
print('As 20 palavras mais associadas a reviews POSITIVAS:')
print(top_positive_words)

print('\nAs 20 palavras mais associadas a reviews NEGATIVAS:')
print(top_negative_words)

As 20 palavras mais associadas a reviews POSITIVAS:
       Palavras:      Coef
289        antes  7.087625
1975   excelente  5.478253
4605        timo  5.472375
4254       rpida  4.956372
4261       rpido  4.715341
4602        tima  4.399510
3318     parabns  4.345467
4758        tudo  4.287822
614          bom  4.274661
97        adorei  4.270913
2697       lindo  4.162540
3430    perfeito  4.122076
3635       prazo  4.033829
228         amei  4.022248
4318  satisfeito  3.813753
567          bem  3.684423
595          boa  3.640198
3270       otimo  3.604786
4046   recomendo  3.559001
4317  satisfeita  3.518553

As 20 palavras mais associadas a reviews NEGATIVAS:
         Palavras:      Coef
3884        pssima -4.994929
3885        pssimo -3.897196
3363        passou -3.421900
3097           nao -3.353226
509          baixa -3.302478
4273          ruim -3.217409
2441      inferior -3.193767
1034       comprei -3.122944
4011        recebi -3.083151
131     aguardando -3.065866
1167     